In [72]:
import tensorflow as tf
print("Using TensorFlow version %s" % tf.__version__)

Using TensorFlow version 2.5.0


In [73]:
import numpy as np

##12.2 넘파이처럼 텐서플로 사용하기

###12.2.1 텐서와 연산

In [74]:
tf.constant([[1., 2., 3.],
             [4., 5., 6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [75]:
# tf.convert_to_tensor
_array_with_reshape = np.arange(1, 7, dtype=np.float32).reshape((2, -1))
tf.convert_to_tensor(_array_with_reshape)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [76]:
_array = np.arange(1, 7, dtype=np.float32)
# tf.convert_to_tensor
_tf_array = tf.convert_to_tensor(_array)
# tf.reshape
_tf_reshaped = tf.reshape(_tf_array, (2, 3))
_tf_reshaped

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [77]:
tf.experimental.numpy?

In [78]:
# dtype, shape
_tf_reshaped.dtype, _tf_reshaped.shape

(tf.float32, TensorShape([2, 3]))

In [79]:
# indexing
_tf_reshaped[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [80]:
# tf.newaxis
_tf_reshaped[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

###12.2.2 텐서와 넘파이

In [81]:
# numpy array to tensor
a = np.array([2, 4, 5], dtype=np.float64)
t = tf.constant(a)
t

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [82]:
# tensor to numpy array
t.numpy()

array([2., 4., 5.])

###12.2.3 타입 변환

In [83]:
t.dtype

tf.float64

In [84]:
# tf.cast?

In [85]:
t = tf.cast(t, tf.float32) # inplace=False
t.dtype

tf.float32

###12.2.4 변수

In [86]:
_arr = np.arange(1, 7, dtype=np.float32).reshape((2, -1))
v = tf.Variable(_arr, name='test_variable')
v

<tf.Variable 'test_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [87]:
v * 2
v

<tf.Variable 'test_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [88]:
# assign ; inplace=True
v.assign(v * 2)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [89]:
# indexing하여 assign
# broadcasting❌
v[:, 2].assign([77, 77])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4., 77.],
       [ 8., 10., 77.]], dtype=float32)>

###12.2.5 다른 데이터 구조


* 희소 텐서
* 텐서 배열
* 래그드 텐서
* 문자열 텐서
* 집합
* 큐

##12.3 사용자 정의 모델과 훈련 알고리즘

###12.3.1 사용자 정의 손실 함수

In [90]:
tf.keras.losses.Huber?

In [91]:
# tf.where?

In [92]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - .5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [93]:
# huber function standalone usage
np.random.seed(777)
y_true = np.random.randint(0, 5, 10)
y_true = y_true.astype(np.float64)
error = np.random.rand(10) * 2
y_pred = y_true + error
huber_fn(tf.constant(y_true), tf.constant(y_pred))

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([0.01269968, 0.67922751, 0.23576545, 1.4777523 , 0.75294641,
       0.86355856, 0.60451363, 0.14457146, 0.27864514, 0.09939387])>

In [94]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - threshold**2*0.5
        return tf.where(is_small_error, squared_loss, linear_loss)

In [95]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - self.threshold**2*0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}

In [96]:
hloss = HuberLoss(name='hloss')

In [97]:
hloss.get_config()

{'name': 'hloss', 'reduction': 'auto', 'threshold': 1.0}

In [98]:
def func(a, b, c):
    return a + b * c

In [99]:
abc_params = {'a':1, 'b':2, 'c':3}

In [100]:
func(**abc_params)

7

###12.3.3 활성화 함수, 초기화, 규제, 제한을 커스터마이징하기

In [101]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [102]:
_weights = np.random.randint(1,10, 12).reshape([2, 2, 3])
_weights

array([[[8, 9, 1],
        [9, 4, 3]],

       [[1, 4, 4],
        [5, 1, 7]]])

In [103]:
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"facotr":self.factor}

In [104]:
my_l1_regularizer = MyL1Regularizer(2.)
my_l1_regularizer.__call__(_weights)

<tf.Tensor: shape=(), dtype=float64, numpy=112.0>

In [105]:
my_l1_regularizer.get_config()

{'facotr': 2.0}

###12.3.4 사용자 정의 지표

In [106]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1., **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}

In [107]:
tf.keras.metrics.Metric.add_weight?

###12.3.5 사용자 정의 층

In [108]:
# 가중치가 없는 레이어
exp_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))
exp_layer(tf.constant(0.))

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [109]:
tf.keras.activations.get('sigmoid')

<function tensorflow.python.keras.activations.sigmoid>

In [110]:
# 가중치를 갖는 레이어: MyDense
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)
    
    def call(self, X):
        return self.activation(X @ self.kernerl + self.bias) 
                                # @; matmul

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
                              # input_shape                       # output_shape
    
    def get_config(self):
        base_config = super().__init__(**kwargs)
        return {**base_config, 
                "units":self.units,
                "activation":tf.keras.activations.serialize(self.activation)}
                                                  # 활성함수 전체설정 저장

In [111]:
tf.keras.layers.Layer.add_weight?

In [112]:
tf.constant(np.arange(1, 7).reshape((3, 2))).shape.as_list()[:-1]+[10]
                                            # input_shape        # output_shape

[3, 10]

In [113]:
tf.keras.layers.GaussianNoise

tensorflow.python.keras.layers.noise.GaussianNoise

###12.3.6 사용자 정의 모델

In [114]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation='elu',
                                           kernel_initializer='he_normal')
        for _ in range(n_layers)]
    
    def call(self, input):
        Z = inputs # resiudal
        for layer in self.hidden:
            Z = layer(Z)
        Z += inputs
        return Z

In [115]:
class ResidualRegressor(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation='elu', 
                                             kernel_initializer='he_normal')
        self.block1 = ResidualBlcok(2, 30)
        self.block2 = ResidualBlcok(2, 30)
        self.out = tf.keras.layers.Dense(output_dim)
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1+3):
            Z = block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

###12.3.7 모델 구성 요소에 기반한 손실과 지표

In [116]:
class ReconstructionRegressor(tf.keras.Model):
    def __init__(self, output_dims, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = [tf.keras.layers.Dense(30, activation='selu', 
                                              kernel_initializer='lecun_normal') 
        for _ in range(5)]
        self.out = tf.keras.layers.Dense(output_dims)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruction = tf.keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden1:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(inputs - reconstruction))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)

###12.3.8 자동 미분을 사용하여 그레이디언트 계산하기

In [117]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

In [118]:
w1, w2 = 5, 3

In [119]:
eps = 1e-6

In [120]:
(f(w1+eps, w2) - f(w1, w2)) / eps

36.000003007075065

In [121]:
(f(w1, w2+eps) - f(w1, w2)) / eps

10.000000003174137

In [122]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)

In [123]:
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [124]:
def f(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

In [125]:
with tf.GradientTape() as tape:
    z = f(w1, w2)

In [126]:
gradients = tape.gradient(z, [w1, w2])

###12.3.9 사용자 정의 훈련 반복

In [127]:
l2_reg = tf.keras.regularizers.l2(0.05)
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(30, activation='elu', 
                                                          kernel_initializer='he_normal',
                                                          kernel_regularizer=l2_reg),
                                    tf.keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

In [128]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [129]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = ' - '.join([f"{m.name}: {m.result():.4f}"
    for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print(f"\r{iteration}/{total} " + metrics, end=end)

In [130]:
(X_train, y_train), (X_valid, y_valid) = tf.keras.datasets.boston_housing.load_data()

In [131]:
n_epochs = 20
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = tf.keras.losses.mean_squared_error
mean_loss = tf.keras.metrics.Mean()
metrics = [tf.keras.metrics.MeanAbsoluteError()]

In [132]:
for epoch in range(1, n_epochs+1):
    print(f"Epoch {epoch}/{n_epochs}")
    for step in range(1, n_steps+1):
        X_batch, y_batch = random_batch(X_train, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_state()

Epoch 1/20
404/404 mean: 27080.5000 - mean_absolute_error: 128.7218
Epoch 2/20
404/404 mean: 1406.1182 - mean_absolute_error: 34.9884
Epoch 3/20
404/404 mean: 362.4258 - mean_absolute_error: 16.9028
Epoch 4/20
404/404 mean: 198.4384 - mean_absolute_error: 10.3310
Epoch 5/20
404/404 mean: 135.6058 - mean_absolute_error: 8.2251
Epoch 6/20
404/404 mean: 115.3766 - mean_absolute_error: 7.5894
Epoch 7/20
404/404 mean: 111.3799 - mean_absolute_error: 7.4236
Epoch 8/20
404/404 mean: 110.7045 - mean_absolute_error: 7.8496
Epoch 9/20
404/404 mean: 114.5502 - mean_absolute_error: 8.0454
Epoch 10/20
404/404 mean: 103.9846 - mean_absolute_error: 7.6020
Epoch 11/20
404/404 mean: 110.1575 - mean_absolute_error: 7.8947
Epoch 12/20
404/404 mean: 97.3473 - mean_absolute_error: 7.3531
Epoch 13/20
404/404 mean: 90.8176 - mean_absolute_error: 6.8918
Epoch 14/20
404/404 mean: 114.9668 - mean_absolute_error: 7.9309
Epoch 15/20
404/404 mean: 107.0367 - mean_absolute_error: 7.5707
Epoch 16/20
404/404 mean: 11

## 12.4 텐서플로 함수와 그래프

In [133]:
def cube(x):
    return x**3

In [134]:
cube(2)
cube(tf.constant(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [135]:
# 텐서플로 함수로 변환: tf.function
tf_cube = tf.function(cube)
tf_cube

In [136]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [137]:
@tf.function
def tf_cube(x):
    return x ** 3

In [138]:
# 원본 파이썬 함수가 필요할 때: .python_function 메서드
tf_cube.python_function(2)

8

##12.5 연습문제

In [142]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [141]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>